In [ ]:
import nnabla as nn

import nnabla.functions as F
import nnabla.parametric_functions as PF
import nnabla.solvers as S
from nnabla.monitor import tile_images

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

%matplotlib inline

from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
mnist = input_data.read_data_sets('../../MNIST_data', one_hot=True)
mb_size = 64
Z_dim = 100
X_dim = mnist.train.images.shape[1]
y_dim = mnist.train.labels.shape[1]
lr = 1e-3

In [ ]:
def G(z, c, hidden=[128, 128]):
    hs = []
    with nn.parameter_scope("G"):  # Parameter scope can be nested
        h = F.concatenate(z, c)
        for hid, hsize in enumerate(hidden):
            with nn.parameter_scope("affine{}".format(hid + 1)):
                h = F.relu(PF.affine(h, hsize))
                hs.append(h)
        with nn.parameter_scope("sigmoid_layer"):
            X = F.sigmoid(PF.affine(h, X_dim))
    return X

In [ ]:
def D(X, c, hidden=[128, 128]):
    hs = []
    with nn.parameter_scope("D"):  # Parameter scope can be nested
        h = F.concatenate(X, c)
        for hid, hsize in enumerate(hidden):
            with nn.parameter_scope("affine{}".format(hid + 1)):
                h = F.relu(PF.affine(h, hsize))
                hs.append(h)
        with nn.parameter_scope("classifier"):
            y = F.sigmoid(PF.affine(h, 1))
    return y

In [ ]:
G_solver = S.Adam(lr)
with nn.parameter_scope("G"):
    G_solver.set_parameters(nn.get_parameters())
    
D_solver = S.Adam(lr)
with nn.parameter_scope("D"):
    D_solver.set_parameters(nn.get_parameters())

In [ ]:
def reset_grad():
    G_solver.zero_grad()
    D_solver.zero_grad()

In [ ]:
def show16(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    plt.show()

In [ ]:
ones_label = nn.Variable.from_numpy_array(np.ones((mb_size, 1)))
zeros_label = nn.Variable.from_numpy_array(np.zeros((mb_size, 1)))

for it in range(1000000):
    # Discriminator
    z = nn.Variable.from_numpy_array(np.random.randn(mb_size, Z_dim))
    X, c = mnist.train.next_batch(mb_size)
    X = nn.Variable.from_numpy_array(X)
    c = nn.Variable.from_numpy_array(c)
    
    G_sample = G(z, c)
    D_real = D(X, c)
    D_fake = D(G_sample, c)

    D_loss_real = F.binary_cross_entropy(D_real, ones_label)
    D_loss_fake = F.binary_cross_entropy(D_fake, zeros_label)
    D_loss = D_loss_real + D_loss_fake
    
    D_loss.forward()
    D_loss.backward()
    D_solver.update()
    
    reset_grad()

    # Generator Update
    z = nn.Variable.from_numpy_array(np.random.randn(mb_size, Z_dim))
    G_sample = G(z, c)
    D_fake = D(G_sample, c)

    G_loss = F.binary_cross_entropy(D_fake, ones_label)

    G_loss.forward()
    G_loss.backward()
    G_solver.update()

    reset_grad()

    # Generate and Show Samples 
    if it % 1000 == 0:
        print('Step: {}, D_loss: {},  G_loss: {}'.format(it, D_loss.d[0], G_loss.d[0]))
        
        c = np.zeros(shape=[mb_size, y_dim], dtype='float32')
        c[:, np.random.randint(0, 10)] = 1.
        c = nn.Variable.from_numpy_array(c)
        samples = G(z, c).d[:16]
        show16(samples)